In [1]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

/Users/pdh21/anaconda3/envs/xidplus/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_GAMA-15/data/holes_GAMA-15_ukidss_k_O16_20180327.fits')

## Read in CIGALE predictions catalogue

In [4]:
cigale=Table.read('../GAMA15_Ldust_prediction_results.fits')

In [5]:
cigale['id'].name = 'help_id'

## Read in photoz

In [6]:
photoz=Table.read('../../dmu24/dmu24_GAMA-15/data/master_catalogue_gama-15_20180119_photoz_20180210_r_optimised.fits')

In [7]:
photoz

help_id,RA,DEC,id,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,za_hb,chi_r_eazy,chi_r_atlas,chi_r_cosmos,chi_r_stellar,stellar_type
bytes27,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes6
HELP_J141546.547+000529.927,213.94394443384806,0.09164634854640615,322065,0.9028,0.583,1.2407,0.78,1.6682,1.642,1.698,0.019,0.8889996515380697,0.0038901725,0.036354,0.070896275,1.8363205,m4iii
HELP_J144113.844-000720.633,220.3076845382463,-0.12239815588151215,322066,1.1282,0.5225,1.856,0.775,0.3912,0.3587,0.4212,0.021,0.8609178972663641,0.067539125,0.034600075,0.01283038,1.328487,f8v
HELP_J143631.813+002833.098,219.13255328515575,0.475860557928688,322067,1.29,0.7952,1.8991,0.796,-99.0,-99.0,-99.0,-99.0,1.048127879735366,0.0010802005,0.01882907,0.0368701,9.1299375,m1iii
HELP_J143302.417+002405.826,218.2600711193127,0.4016182827662753,322068,1.1347,0.3875,1.9517,0.798,-99.0,-99.0,-99.0,-99.0,1.0056287119426772,-99.0,-99.0,-99.0,-99.0,
HELP_J144038.334-002408.434,220.1597268871738,-0.40234281903922925,322069,1.0148,0.6858,1.4809,0.775,1.7087,1.6659,1.7552,0.024,0.8665006509581632,0.000401907,0.125252075,0.29048875,12.5883675,wg8iii
HELP_J143418.946-004636.350,218.5789415174339,-0.7767638219889839,322070,0.886,0.5089,1.2677,0.702,1.7168,1.5872,1.856,0.073,0.8833496027298801,0.2708705,0.57580525,0.44484775,6.14024,wk2iii
HELP_J143159.128-000214.789,217.99636806811705,-0.03744126534387154,322071,0.8571,0.3958,1.2813,0.641,1.6539,1.3862,1.9165,0.156,0.8833496027298801,0.041801725,0.573121,0.220772575,2.48693275,wk2iii
HELP_J144020.768-002750.029,220.0865319196699,-0.4638968494879676,322072,0.9088,0.6908,1.2207,0.796,-99.0,-99.0,-99.0,-99.0,0.8721001529110377,0.11123585,0.70335325,0.6111925,18.1824775,wk2iii
HELP_J143048.452+010925.488,217.7018829344781,1.1570799647325665,322073,0.9517,0.6262,1.2677,0.796,-99.0,-99.0,-99.0,-99.0,0.8889996515380697,0.1052213,0.85642575,0.41004575,17.4037275,wk2iii


## Join CIGALE and photoz tables

In [8]:
prior=join(cigale,photoz)

## Import model/s to find f250_pred


In [9]:
# We have fit a model to the data on other fields to obtained an empirical relationship
# between the ratio (Ldust/f250) and redshift

import pickle

# Function used to fit the data (exponential)
def func(p, x):
    p1, p2 = p
    return p1*np.log(x)+p2

# read model (fitted to all data - XID+)
c_model1a = pickle.load(open('../Ldust_investigation/cosmos_model_usingChi2.pkl', 'rb'))

## Find Ldust/f250 ratio relationship for each model

In [10]:
# ## NEW MODELS - XID+ ##
#######################

# Read model and apply ratio - z function to the dataset
logLflux_ratioC = func(c_model1a.beta, prior['z1_median'])
# Extract the flux prediction from the ratio (Ldust/f250) relationship
Lflux_ratioC = 10**logLflux_ratioC
f_pred = prior['bayes.dust.luminosity'] / Lflux_ratioC


In [11]:
# Different flux cuts
# After an analysis running XID+ on a few tiles with different flux cuts, 
# we decided to use f_pred > 5mJy as our flux cut
print('len total prior list: ', len(prior))
print('len C1 cut > 0.2: ', len(prior[f_pred > 0.2]))
print('len C1 cut > 1: ', len(prior[f_pred > 1]))
print('len C1 cut > 2: ', len(prior[f_pred > 2]))
print('len C1 cut > 3: ', len(prior[f_pred > 3]))
print('len C1 cut > 4: ', len(prior[f_pred > 4]))
print('len C1 cut > 5: ', len(prior[f_pred > 5]))
print('len C1 cut > 7: ', len(prior[f_pred > 7]))
print('len C1 cut > 10: ', len(prior[f_pred > 10]))
print('len C1 cut > 20: ', len(prior[f_pred > 20]))
print('len C1 cut > 50: ', len(prior[f_pred > 50]))
print('len C1 cut > 100: ', len(prior[f_pred > 100]))

len total prior list:  1726717
len C1 cut > 0.2:  1726396
len C1 cut > 1:  1725948
len C1 cut > 2:  1724096
len C1 cut > 3:  1717011
len C1 cut > 4:  1700498
len C1 cut > 5:  1669694
len C1 cut > 7:  1578557
len C1 cut > 10:  1434052
len C1 cut > 20:  1090885
len C1 cut > 50:  605015
len C1 cut > 100:  295333


In [12]:
prior_sel = prior[['help_id', 'RA', 'DEC']]
prior_sel['f250_pred'] = f_pred
prior_sel['DEC'].name='Dec'
prior_sel=prior_sel[f_pred>5]

In [13]:
prior_sel

help_id,RA,Dec,f250_pred
bytes27,float64,float64,float64
HELP_J140011.281+012223.464,210.04700380700902,1.3731843653253262,27.30008402913021
HELP_J140011.780+012459.432,210.04908433885802,1.4165089588858322,198.56400360488286
HELP_J140012.393+012309.583,210.05163616447652,1.3859951494563272,134.15493823556724
HELP_J140012.790+012311.829,210.0532936292695,1.3866190856275589,40.41331087325194
HELP_J140012.839+012417.993,210.05349653945746,1.4049981030735224,16.313739001690557
HELP_J140012.859+012656.739,210.05357924856725,1.449094303505662,119.18346714062172
HELP_J140012.887+012644.810,210.05369404812356,1.4457804182051432,32.080835848886196
HELP_J140013.203+012516.958,210.0550133505917,1.4213771677872309,584.9652686875751
HELP_J140013.430+012310.483,210.05595847576842,1.3862452235271698,36.56166381092505


In [14]:
prior['DEC'].name='Dec'

## Read in Maps

In [15]:
pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-15_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-15_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/GAMA-15_SPIRE500_v1.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [16]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3 #convert to mJy
nim250=hdulist[3].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3 #convert to mJy
nim350=hdulist[3].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3 #convert to mJy
nim500=hdulist[3].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cdelt #pixel size (in arcseconds)
hdulist.close()

In [17]:
## Set XID+ prior class

In [18]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior_sel['RA'] ,prior_sel['Dec'] ,'GAMA15_Ldust_prediction_results.fits',ID=prior_sel['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior_sel['RA'] ,prior_sel['Dec'] ,'GAMA15_Ldust_prediction_results.fits',ID=prior_sel['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior_sel['RA'] ,prior_sel['Dec'] ,'GAMA15_Ldust_prediction_results.fits',ID=prior_sel['help_id'] )
prior500.prior_bkg(-5.0,5)

In [19]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0,1] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1,1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2,1] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [20]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 4940 tiles required for input catalogue and 101 large tiles
writing total_bytes=1320401211...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 1320401211)... done.


SystemExit: 

In [ ]:
prior250.nsrc